In [1]:
# Load data preprocessing libs
import pandas as pd   # dataset
import numpy as np  # maths
from datetime import datetime,date # date time lib

import re
from bs4 import BeautifulSoup  # removing html tags

# Load vectorizer and similarity measure
from sklearn.feature_extraction.text import TfidfVectorizer  # term freq inverse doc. freq.
from sklearn.metrics.pairwise import cosine_similarity  # measuring the similarity of text

import tkinter   # User interface
import threading
from tkinter import *
from tkinter import messagebox
import speech_recognition as speech # taking input through voice
import import_ipynb
# from mailing import mail

In [2]:
q = pd.read_csv("Dataset/Questions.csv",encoding='latin-1')
an = pd.read_csv("Dataset/AnswersV2.csv",encoding='latin-1')

In [3]:
an.head()

,AId,OwnerUserId,CreationDate,QId,Score,Body,latest_score,alternate,date
0,536,161.0,2008-08-02T18:49:07Z,502,9,<p>You can use ImageMagick's convert utility f...,9,9,2022-07-08
1,538,156.0,2008-08-02T18:56:56Z,535,23,<p>One possibility is Hudson. It's written in...,23,23,2022-07-08
2,541,157.0,2008-08-02T19:06:40Z,535,20,"<p>We run <a href=""http://buildbot.net/trac"">B...",20,20,2022-07-08
3,595,116.0,2008-08-03T01:17:36Z,594,25,<p>The canonical way is to use the built-in cu...,25,25,2022-07-08
4,660,197.0,2008-08-03T12:09:18Z,535,14,<p>Second the Buildbot - Trac integration. You...,14,14,2022-07-08


In [4]:
an.rename(columns={'ParentId':'QId'},inplace=True)
an.head()

,AId,OwnerUserId,CreationDate,QId,Score,Body,latest_score,alternate,date
0,536,161.0,2008-08-02T18:49:07Z,502,9,<p>You can use ImageMagick's convert utility f...,9,9,2022-07-08
1,538,156.0,2008-08-02T18:56:56Z,535,23,<p>One possibility is Hudson. It's written in...,23,23,2022-07-08
2,541,157.0,2008-08-02T19:06:40Z,535,20,"<p>We run <a href=""http://buildbot.net/trac"">B...",20,20,2022-07-08
3,595,116.0,2008-08-03T01:17:36Z,594,25,<p>The canonical way is to use the built-in cu...,25,25,2022-07-08
4,660,197.0,2008-08-03T12:09:18Z,535,14,<p>Second the Buildbot - Trac integration. You...,14,14,2022-07-08


In [5]:
q.rename(columns={'Id':'QId'},inplace=True)
an.rename(columns={'Id':'AId'},inplace=True)

In [6]:
an = an[an['Score']>5]
an.head()

,AId,OwnerUserId,CreationDate,QId,Score,Body,latest_score,alternate,date
0,536,161.0,2008-08-02T18:49:07Z,502,9,<p>You can use ImageMagick's convert utility f...,9,9,2022-07-08
1,538,156.0,2008-08-02T18:56:56Z,535,23,<p>One possibility is Hudson. It's written in...,23,23,2022-07-08
2,541,157.0,2008-08-02T19:06:40Z,535,20,"<p>We run <a href=""http://buildbot.net/trac"">B...",20,20,2022-07-08
3,595,116.0,2008-08-03T01:17:36Z,594,25,<p>The canonical way is to use the built-in cu...,25,25,2022-07-08
4,660,197.0,2008-08-03T12:09:18Z,535,14,<p>Second the Buildbot - Trac integration. You...,14,14,2022-07-08


In [7]:
q.head()

,QId,OwnerUserId,CreationDate,Score,Title,Body
0,469,147.0,2008-08-02T15:11:16Z,21,How can I find the full path to a font from it...,<p>I am using the Photoshop's javascript API t...
1,502,147.0,2008-08-02T17:01:58Z,27,Get a preview JPEG of a PDF on Windows?,<p>I have a cross-platform (Python) applicatio...
2,535,154.0,2008-08-02T18:43:54Z,40,Continuous Integration System for a Python Cod...,<p>I'm starting work on a hobby project with a...
3,594,116.0,2008-08-03T01:15:08Z,25,cx_Oracle: How do I iterate over a result set?,<p>There are several ways to iterate over a re...
4,683,199.0,2008-08-03T13:19:16Z,28,Using 'in' to match an attribute of Python obj...,<p>I don't remember whether I was dreaming or ...


In [8]:
df = q.merge(an, on='QId')
df.head()

,QId,OwnerUserId_x,CreationDate_x,Score_x,Title,Body_x,AId,OwnerUserId_y,CreationDate_y,Score_y,Body_y,latest_score,alternate,date
0,469,147.0,2008-08-02T15:11:16Z,21,How can I find the full path to a font from it...,<p>I am using the Photoshop's javascript API t...,3040,457.0,2008-08-06T03:01:23Z,12,<p>Unfortunately the only API that isn't depre...,12,12,2022-07-08
1,502,147.0,2008-08-02T17:01:58Z,27,Get a preview JPEG of a PDF on Windows?,<p>I have a cross-platform (Python) applicatio...,536,161.0,2008-08-02T18:49:07Z,9,<p>You can use ImageMagick's convert utility f...,9,9,2022-07-08
2,502,147.0,2008-08-02T17:01:58Z,27,Get a preview JPEG of a PDF on Windows?,<p>I have a cross-platform (Python) applicatio...,7090,13.0,2008-08-10T08:08:33Z,25,<p>ImageMagick delegates the PDF->bitmap conve...,25,25,2022-07-08
3,535,154.0,2008-08-02T18:43:54Z,40,Continuous Integration System for a Python Cod...,<p>I'm starting work on a hobby project with a...,538,156.0,2008-08-02T18:56:56Z,23,<p>One possibility is Hudson. It's written in...,23,23,2022-07-08
4,535,154.0,2008-08-02T18:43:54Z,40,Continuous Integration System for a Python Cod...,<p>I'm starting work on a hobby project with a...,541,157.0,2008-08-02T19:06:40Z,20,"<p>We run <a href=""http://buildbot.net/trac"">B...",20,20,2022-07-08


In [9]:
df.drop(columns=['OwnerUserId_x','CreationDate_x','Score_x','OwnerUserId_y','CreationDate_y'],inplace=True)
df.head()

,QId,Title,Body_x,AId,Score_y,Body_y,latest_score,alternate,date
0,469,How can I find the full path to a font from it...,<p>I am using the Photoshop's javascript API t...,3040,12,<p>Unfortunately the only API that isn't depre...,12,12,2022-07-08
1,502,Get a preview JPEG of a PDF on Windows?,<p>I have a cross-platform (Python) applicatio...,536,9,<p>You can use ImageMagick's convert utility f...,9,9,2022-07-08
2,502,Get a preview JPEG of a PDF on Windows?,<p>I have a cross-platform (Python) applicatio...,7090,25,<p>ImageMagick delegates the PDF->bitmap conve...,25,25,2022-07-08
3,535,Continuous Integration System for a Python Cod...,<p>I'm starting work on a hobby project with a...,538,23,<p>One possibility is Hudson. It's written in...,23,23,2022-07-08
4,535,Continuous Integration System for a Python Cod...,<p>I'm starting work on a hobby project with a...,541,20,"<p>We run <a href=""http://buildbot.net/trac"">B...",20,20,2022-07-08


In [10]:
df.rename(columns={'Body_x':'Question','Body_y':'Answer','Score_y':'Score'},inplace=True)

In [11]:
df.head()

,QId,Title,Question,AId,Score,Answer,latest_score,alternate,date
0,469,How can I find the full path to a font from it...,<p>I am using the Photoshop's javascript API t...,3040,12,<p>Unfortunately the only API that isn't depre...,12,12,2022-07-08
1,502,Get a preview JPEG of a PDF on Windows?,<p>I have a cross-platform (Python) applicatio...,536,9,<p>You can use ImageMagick's convert utility f...,9,9,2022-07-08
2,502,Get a preview JPEG of a PDF on Windows?,<p>I have a cross-platform (Python) applicatio...,7090,25,<p>ImageMagick delegates the PDF->bitmap conve...,25,25,2022-07-08
3,535,Continuous Integration System for a Python Cod...,<p>I'm starting work on a hobby project with a...,538,23,<p>One possibility is Hudson. It's written in...,23,23,2022-07-08
4,535,Continuous Integration System for a Python Cod...,<p>I'm starting work on a hobby project with a...,541,20,"<p>We run <a href=""http://buildbot.net/trac"">B...",20,20,2022-07-08


In [12]:
df['Answer'] = df['Answer'].apply(lambda x:BeautifulSoup(x).get_text())
df['Question'] = df['Question'].apply(lambda x:BeautifulSoup(x).get_text())

In [13]:
df.head()

,QId,Title,Question,AId,Score,Answer,latest_score,alternate,date
0,469,How can I find the full path to a font from it...,I am using the Photoshop's javascript API to f...,3040,12,Unfortunately the only API that isn't deprecat...,12,12,2022-07-08
1,502,Get a preview JPEG of a PDF on Windows?,I have a cross-platform (Python) application w...,536,9,You can use ImageMagick's convert utility for ...,9,9,2022-07-08
2,502,Get a preview JPEG of a PDF on Windows?,I have a cross-platform (Python) application w...,7090,25,ImageMagick delegates the PDF->bitmap conversi...,25,25,2022-07-08
3,535,Continuous Integration System for a Python Cod...,I'm starting work on a hobby project with a py...,538,23,One possibility is Hudson. It's written in Ja...,23,23,2022-07-08
4,535,Continuous Integration System for a Python Cod...,I'm starting work on a hobby project with a py...,541,20,"We run Buildbot - Trac at work, I haven't used...",20,20,2022-07-08


In [14]:
vectorizer = TfidfVectorizer()
vectorizer.fit(np.concatenate((df.Question, df.Answer)))

TfidfVectorizer()

In [15]:
Question_vectors = vectorizer.transform(df.Question)

In [16]:
threshold = 30
answer = []

In [17]:
def getAnswerWithHighestScore(answers):
    r = np.argmax(answers['latest_score']-answers['alternate'])
    if (datetime.strptime(str(date.today()),"%Y-%m-%d")-datetime.strptime(answers['date'].iloc[0],"%Y-%m-%d")).days > threshold:
        df[df['QId'] == answers['QId'].iloc[0]]['date'] = date.today()
        df[df['QId'] == answers['QId'].iloc[0]]['alternate']=df[df['QId'] == answers['QId'].iloc[0]]['latest_score']
        an[an['QId'] == answers['QId'].iloc[0]]['date'] = date.today()
        an[an['QId'] == answers['QId'].iloc[0]]['alternate']=df[df['QId'] == answers['QId'].iloc[0]]['latest_score']
        an.to_csv("Dataset/AnswersV2.csv",index=False)
    return [answers.iloc[r]['Answer'],answers.iloc[r]['AId'],answers.iloc[r]['alternate']]

In [18]:
def get_answer(row):
    global answer
    
    qid = df.iloc[row][0]  # question id similar to the question asked by user
    answers = df.loc[df['QId'] == qid]
    answer = getAnswerWithHighestScore(answers)  # retreive the ans with highest score 
    return [answer[0],answer[2]]

In [19]:
def chatbot_response(msg):
    input_question =BeautifulSoup(msg).get_text()

    # Locate the closest question
    input_question_vector = vectorizer.transform([input_question])

    # Compute similarities
    similarities = cosine_similarity(input_question_vector, Question_vectors)

    # Find the closest question
    closest = np.argmax(similarities, axis=1)
    a =  get_answer(closest[0])
    return [a[0],a[1]]
#     return df.Answer.iloc[closest].values[0]

In [20]:
global check,upvoted,downvoted
check = True
upvoted = False
downvoted = False

In [21]:
def upvote():
    global upvoted,downvoted
    if not upvoted:
        upvoted = True
        downvoted = False
        r1 = df.index[df['AId'] == answer[1]]
        r2 = an.index[df['AId'] == answer[1]]
        score = int(df.iloc[r1]['latest_score'].values[0])
        score+=1
        updateScore(score)
        df.loc[r1,'latest_score'] = score
        an.loc[r2,'latest_score'] = score
        an.to_csv("Dataset/AnswersV2.csv",index=False)
        messagebox.showinfo("UPVOTE!!!","ANSWER IS UPVOTED")

In [22]:
def downvote():
    global upvoted,downvoted
    if not downvoted:
        downvoted = True
        upvoted = False
        r1 = df.index[df['AId'] == answer[1]]
        r2 = an.index[df['AId'] == answer[1]]
        score = int(df.iloc[r1]['latest_score'].values[0])
        score-=1
        updateScore(score)
#         if(score <0):
#             mail(answer[1])
        df.loc[r1,'latest_score'] = score
        an.loc[r2,'latest_score'] = score
        an.to_csv("Dataset/AnswersV2.csv",index=False)
        messagebox.showinfo("DOWNVOTE!!!","ANSWER IS DOWNVOTED")

In [23]:
def updateScore(newScore):
    score_text.delete("1.0",END)
    score_text.insert("1.0",newScore)

In [25]:
class TrieNode():
    def __init__(self):
        # Initialising one node for trie
        self.children = {}
        self.last = False
        self.answers=[]


class Trie():
    def __init__(self):

        # Initialising the trie structure.
        self.root = TrieNode()
        self.answers = []

    def insert(self, key):

        # Inserts a key into trie if it does not exist already.
        # And if the key is a prefix of the trie node, just
        # marks it as leaf node.
        node = self.root

        for a in key:
            if not node.children.get(a):
                node.children[a] = TrieNode()

            node = node.children[a]

        node.last = True

    def suggestionsRec(self, node, word):

        # Method to recursively traverse the trie
        # and return a whole word.
        if node.last:
            self.answers.append(word)

        for a, n in node.children.items():
            self.suggestionsRec(n, word + a)

    def printAutoSuggestions(self, key):

        # Returns all the words in the trie whose common
        # prefix is the given key thus listing out all
        # the suggestions for autocomplete.
        node = self.root

        for a in key:
            # no string in the Trie has this prefix
            if not node.children.get(a):
                return 0
            node = node.children[a]

        # If prefix is present as a word, but
        # there is no subtree below the last
        # matching node.
        if not node.children:
            return -1

        self.suggestionsRec(node, key)
        return 1

    def getAnswers(self):
        return self.answers

    def reset(self):
        self.answers =[]

In [26]:
que= q.Title.tolist()

In [27]:
ob1 = Trie()
for i in range(30000):
    ob1.insert(que[i])

In [28]:
from pynput.keyboard import Key, Listener

In [29]:
anss = []

In [30]:
import functools

In [31]:
def autoComplete(text=''):
    global anss
    ob1.printAutoSuggestions(text)
    anss = ob1.getAnswers()
    ob1.reset()


In [35]:
def speechToText():
    obj = speech.Recognizer()
    print("PyBot is listening you...")
    with speech.Microphone() as microphone:
        try:
            print("Listening ...")
            audio = obj.listen(microphone)
            print("Processing the Voice ...")
            query = obj.recognize_google(audio,language = 'eng-in')
            print("Recognized the Question ...")
            print(query)
            EntryBox.delete('1.0',END)
            EntryBox.insert('1.0', query)
            send()
        except Exception as e:
            print(e)


def send():
    global upvoted,downvoted
    upvoted = False
    downvoted = False
    msg = EntryBox.get("1.0",'end-1c').strip()
    EntryBox.delete("0.0",END)
    if msg != '':
        ChatLog.config(state=NORMAL)
        ChatLog.insert(END, "You: " + msg + '\n\n')
        ChatLog.config(foreground="#442265", font=("Verdana", 12 ))
        res = chatbot_response(msg)
        ChatLog.insert(END, "Bot: " + res[0] + '\n\n')
        updateScore(res[1])
        ChatLog.config(state=DISABLED)
        ChatLog.yview(END)



base = Tk()
base.title("PyBOT")
base.geometry("600x650")
base.resizable(width=FALSE, height=FALSE)
# base.withdraw()
# def popup(vote):
#     messagebox.showinfo("Alert","Answer was "+vote)

def sp():
    thread = threading.Thread(target=speechToText())
    thread.start()

score_text = Text(base,bg="white")
upvote = Button(base, text="UPVOTE", fg="black",bg="white",command=upvote)
upvote.place(x=30, y=300, height=40, width=90)

downvote = Button(base, text="DOWNVOTE", fg="black",bg="white",command=downvote)
downvote.place(x=180, y=300, height=40, width=90)

speechButton = Button(base, text="Mic", fg="black",bg="white",command=sp)
speechButton.place(x=350, y=300, height=40, width=50)
score_text.place(x=500,y=300, height=40,width=40)

ChatLog = Text(base, bd=0, bg="white", height="8", width="50", font="Arial")
ChatLog.config(state=DISABLED)

scrollbar = Scrollbar(base, command=ChatLog.yview, cursor="heart")
ChatLog['yscrollcommand'] = scrollbar.set

SendButton = Button(base, font=("Verdana",12,'bold'), text="Send", width="12", height=5,
                    bd=0, bg="#32de97", activebackground="#3c9d9b",fg='#ffffff',
                    command= send )

EntryBox = Text(base, bd=0, bg="white",width="29", height="5", font="Arial")

ListBox = Listbox(base, bg="white",width="29", height="5", font="Arial")

scrollbar.place(x=575,y=6, height=280)
ChatLog.place(x=6,y=6, height=280, width=800)
EntryBox.place(x=128, y=380, height=120, width=500)
SendButton.place(x=6, y=380, height=120)
ListBox.place(x=6, y=520, height=100,width = 600)

def update(data):
	# Clear the listbox
	ListBox.delete(0, END)

	for item in data:
		ListBox.insert(END, item)

# Update entry box with listbox clicked
def populate(e):
	# Delete whatever is in the entry box
	EntryBox.delete("1.0", END)

	# Add clicked list item to entry box
	EntryBox.insert("1.0", ListBox.get(ANCHOR))

def getCompleted(event):
    msg = EntryBox.get("1.0",'end-1c').strip()

    autoComplete(msg)
    update(anss[:5])

ListBox.bind("<<ListboxSelect>>", populate)
base.bind("<Key>",getCompleted)
base.mainloop()

C:\Users\win 10\AppData\Local\Temp\ipykernel_14012\329111576.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[df['QId'] == answers['QId'].iloc[0]]['date'] = date.today()
C:\Users\win 10\AppData\Local\Temp\ipykernel_14012\329111576.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[df['QId'] == answers['QId'].iloc[0]]['alternate']=df[df['QId'] == answers['QId'].iloc[0]]['latest_score']
C:\Users\win 10\AppData\Local\Temp\ipykernel_14012\329111576.py:6: SettingWithCopyWarning: 
A value is trying to b